In [1]:
import pandas as pd 
import numpy as np
import sklearn as sklearn
import os as os

import matplotlib.pyplot as plt
import seaborn as sns
from kneed import KneeLocator
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
import re as re

if os.getlogin()=="JVARGH7":
    path_equity_precision_llm_folder = "C:/Cloud/OneDrive - Emory University/Papers/Global Equity in Diabetes Precision Medicine LLM"
    path_equity_precision_llm_repo =  'C:/code/external/equity_precision_llm'

elif os.getlogin()=='aamnasoniwala':
    path_equity_precision_llm_folder = "/Users/aamnasoniwala/Library/CloudStorage/OneDrive-Emory/Global Equity in Diabetes Precision Medicine LLM"
    path_equity_precision_llm_repo = '/Users/aamnasoniwala/code/equity_precision_llm'

excel_path = path_equity_precision_llm_folder + "/llm training/Methods.xlsx"
# path_equity_precision_llm_repo = os.path.abspath("").replace("preprocessing", "")

execfile(path_equity_precision_llm_repo + "/functions/crosstab_summary.py")


In [15]:
# path_equity_precision_llm_folder = "/Users/aamnasoniwala/Library/CloudStorage/OneDrive-Emory/Global Equity in Diabetes Precision Medicine LLM"
# path_equity_precision_llm_repo = '/Users/aamnasoniwala/code/equity_precision_llm'

excel_path_training = path_equity_precision_llm_folder + "/llm training/Methods.xlsx"
excel_path_development = path_equity_precision_llm_folder + "/llm training/epldat03_Development Data.xlsx"
excel_path_test = path_equity_precision_llm_folder + "/llm training/epldat03_Test Data.xlsx"
# path_equity_precision_llm_repo = os.path.abspath("").replace("preprocessing", "")

execfile(path_equity_precision_llm_repo + "/functions/clean_input.py")
execfile(path_equity_precision_llm_repo + "/functions/crosstab_summary.py")

In [16]:
input_training = clean_input(input_path = excel_path_training, sheet_name='Training Data')
input_development = clean_input(input_path = excel_path_development, sheet_name='Sheet1')
input_test = clean_input(input_path = excel_path_test, sheet_name='Sheet1')


In [17]:
input_test

,PMID,Title,Abstract,MeSH,orig_precision_medicine,orig_diabetes,orig_source_population,orig_primary_study,pubmed_source_population
0,23680249,Pattern and predictors of dyslipidemia in pati...,Dyslipidemia is a major risk factor for macro-...,"Adult, Blood Glucose, Cross-Sectional Studies,...",no,yes,sa,yes,SA
1,32585310,Proteome-wide assessment of diabetes mellitus ...,Proteomics is expected to provide novel insigh...,"Adult, Blood Proteins, Case-Control Studies, C...",no,yes,sa,yes,SA
2,34254537,Role of Nutrition Counseling and Lifestyle Mod...,Objective: India is the second country after C...,"Blood Glucose, Counseling, Diabetes Mellitus, ...",no,yes,sa,yes,SA
3,15287926,Lipids and lipoprotein(a) concentrations in Pa...,The aim of the present study was to analyze se...,"Blood Pressure, Case-Control Studies, Diabetes...",no,yes,sa,yes,SA
4,16108841,Telomere shortening occurs in Asian Indian Typ...,Telomere shortening has been reported in sever...,"Aging, Cholesterol, Diabetes Mellitus, Type 2,...",no,yes,sa,yes,SA
...,...,...,...,...,...,...,...,...,...
2076,33786936,Eukaryotic viruses in the fecal virome at the ...,Studies of the fecal virome in type 1 diabetes...,"Adolescent, Azerbaijan, Case-Control Studies, ...",yes,yes,ca,yes,CA
2077,22151254,Genetic variants in potassium channels are ass...,Recent genome-wide association studies (GWAS) ...,"ATP-Binding Cassette Transporters, Alleles, Di...",yes,yes,ca,yes,CA
2078,32870173,The prevalence of major cardiovascular risk fa...,To study the prevalence of cardiovascular (CV)...,"Adolescent, Adult, Aged, Cardiovascular Diseas...",no,no,ca,yes,CA
2079,30255370,"Desertiactinospora gelatinilytica gen. nov., s...","A novel, Gram-positive, spore-forming actinomy...","ATP Binding Cassette Transporter, Subfamily B,...",yes,no,unknown,yes,CA


### Running different scenarios for Training (1 to 4)

In [27]:
combined_output_training = pd.DataFrame()
for scenario in range(1,5):
    results = pd.read_csv(path_equity_precision_llm_folder + '/llm training/Training Scenario '+ str(scenario) +'_results.csv')
    merged_df_training = input_training.merge(results, left_on='PMID', right_on='pmid', how='left')
    merged_df_training['source_population_match'] = merged_df_training.apply(lambda row: bool(re.search('(^|\s)' + str(row['gpt_source_population']),str(row['orig_source_population']))), axis=1)

    # Crosstab summary for precision medicine
    summary_precision_medicine = crosstab_summary(merged_df_training,truth='orig_precision_medicine',test='gpt_precision_medicine')
    summary_diabetes = crosstab_summary(merged_df_training,truth='orig_diabetes',test='gpt_diabetes')
    summary_primary_study = crosstab_summary(merged_df_training,truth='orig_primary_study',test='gpt_primary_study')

    summary_precision_medicine['variable'] = 'Precision Medicine'
    summary_diabetes['variable'] = 'Diabetes'
    summary_primary_study['variable'] = 'Primary Study'  

    t_source_population = pd.crosstab(merged_df_training['source_population_match'], merged_df_training['orig_source_population'])

    prop_correct_source_population = t_source_population.loc[True].sum()/t_source_population.sum().sum() 
    prop_correct_source_population

    summary_source_population = pd.DataFrame({'variable': 'Source Population', 'Accuracy': prop_correct_source_population}, index=[0])


    df_summary = pd.concat([summary_precision_medicine, summary_diabetes, summary_primary_study,summary_source_population])
    df_summary['Scenario'] = 'Scenario' + str(scenario)
    combined_output_training = pd.concat([combined_output_training,df_summary],axis=0,ignore_index=True) 

combined_output_training.to_csv(path_equity_precision_llm_repo + '/preprocessing/epl03_combined output_Training.csv', index=False)

### Running different scenarios for Development (1 to 4)

In [26]:
combined_output_development = pd.DataFrame()
for scenario in range(1,5):
    results = pd.read_csv(path_equity_precision_llm_folder + '/llm training/Development Scenario '+ str(scenario) +'_results.csv')
    merged_df_development = input_development.merge(results, left_on='PMID', right_on='pmid', how='left')
    merged_df_development['source_population_match'] = merged_df_development.apply(lambda row: bool(re.search('(^|\s)' + str(row['gpt_source_population']),str(row['orig_source_population']))), axis=1)

    # Crosstab summary for precision medicine
    summary_precision_medicine = crosstab_summary(merged_df_development,truth='orig_precision_medicine',test='gpt_precision_medicine')
    summary_diabetes = crosstab_summary(merged_df_development,truth='orig_diabetes',test='gpt_diabetes')
    summary_primary_study = crosstab_summary(merged_df_development,truth='orig_primary_study',test='gpt_primary_study')

    summary_precision_medicine['variable'] = 'Precision Medicine'
    summary_diabetes['variable'] = 'Diabetes'
    summary_primary_study['variable'] = 'Primary Study'  

    t_source_population = pd.crosstab(merged_df_development['source_population_match'], merged_df_development['orig_source_population'])

    prop_correct_source_population = t_source_population.loc[True].sum()/t_source_population.sum().sum() 
    prop_correct_source_population

    summary_source_population = pd.DataFrame({'variable': 'Source Population', 'Accuracy': prop_correct_source_population}, index=[0])


    df_summary = pd.concat([summary_precision_medicine, summary_diabetes, summary_primary_study,summary_source_population])
    df_summary['Scenario'] = 'Scenario' + str(scenario)
    combined_output_development = pd.concat([combined_output_development,df_summary],axis=0,ignore_index=True) 

combined_output_development.to_csv(path_equity_precision_llm_repo + '/preprocessing/epl03_combined output_Development.csv', index=False)

### Combining different outputs for Test (1 : n_json_splits)

In [ ]:
n_json_splits = 4
results = pd.DataFrame()
for part in range(1,n_json_splits+1):
    results = pd.concat([results,pd.read_csv(path_equity_precision_llm_folder + '/llm training/Test Data Splits/Test Part '+ str(part) +'_results.csv')])
merged_df_test = input_test.merge(results, left_on='PMID', right_on='pmid', how='left')
merged_df_test['source_population_match'] = merged_df_test.apply(lambda row: bool(re.search('(^|\s)' + str(row['gpt_source_population']),str(row['orig_source_population']))), axis=1)

    # Crosstab summary for precision medicine
summary_precision_medicine = crosstab_summary(merged_df_test,truth='orig_precision_medicine',test='gpt_precision_medicine')
summary_diabetes = crosstab_summary(merged_df_test,truth='orig_diabetes',test='gpt_diabetes')
summary_primary_study = crosstab_summary(merged_df_test,truth='orig_primary_study',test='gpt_primary_study')

summary_precision_medicine['variable'] = 'Precision Medicine'
summary_diabetes['variable'] = 'Diabetes'
summary_primary_study['variable'] = 'Primary Study'  

t_source_population = pd.crosstab(merged_df_test['source_population_match'], merged_df_test['orig_source_population'])


,Sensitivity,Specificity,Accuracy,ppv,npv,variable
0,0.871063,0.864991,0.867527,0.859223,0.876415,Precision Medicine
0,0.923077,0.799599,0.863702,0.834437,0.904762,Diabetes
0,0.933838,0.747664,0.922525,0.985608,0.379147,Primary Study


In [23]:
prop_correct_source_population = t_source_population.loc[True].sum()/t_source_population.sum().sum() 

prop_correct_source_population

0.5624103299856528

In [24]:

summary_source_population = pd.DataFrame({'variable': 'Source Population', 'Accuracy': prop_correct_source_population}, index=[0])


df_summary = pd.concat([summary_precision_medicine, summary_diabetes, summary_primary_study,summary_source_population])


df_summary.to_csv(path_equity_precision_llm_repo + '/preprocessing/epl03_combined output_Test.csv', index=False)

### To review why source population accuracy is low, and for which source populations

In [ ]:
crosstab_result_training = pd.crosstab(merged_df_training['gpt_source_population'], merged_df_training['orig_source_population'])
print(crosstab_result_training)

orig_source_population  ca  cee  ea  lac  mena  na, ssa  sa  seap  ssa  unk  \
gpt_source_population                                                         
cee                      1    1   0    0     1        0   0     0    0    0   
cee, we                  0    1   0    0     0        0   0     0    0    0   
ea                       0    0   1    0     0        0   0     0    0    0   
lac                      0    0   0    2     0        0   0     0    0    0   
mena                     0    0   0    0     1        0   0     0    0    0   
na                       0    0   0    0     0        1   0     0    1    2   
sa                       0    0   0    0     0        0   1     0    0    0   
seap                     0    0   0    0     0        0   0     2    0    0   
we                       0    0   0    0     0        0   0     0    0    0   

orig_source_population  we  
gpt_source_population       
cee                      0  
cee, we                  0  
ea            

In [ ]:
crosstab_result_development = pd.crosstab(merged_df_development['gpt_source_population'], merged_df_development['orig_source_population'])
print(crosstab_result_development)

In [ ]:
crosstab_result_test = pd.crosstab(merged_df_test['gpt_source_population'], merged_df_test['orig_source_population'])
print(crosstab_result_test)